<a href="https://colab.research.google.com/github/iGeology-Illinois/geol-581-module-3-principles-of-uncertainty-chapter-4-seanb7/blob/main/GEOL_581_Module_5_Slope_Stability_Analyses_no1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GEOL 581: Module 5 | Slope Stability Analyses

## Introduction

In this notebook, we will explore how to apply and compare different slope stability approaches, referencing the methods described in the textbook sections on kinematic and mechanical preconditions, plane/rotational sliding, and the method of slices (including Figs. 10.31–10.37, etc.). We will use Python to model:

- Plane sliding in non-cohesive soils (and how friction angle and slope angle interplay).  
- The method of slices in cohesive soils (membrane vs. flow net solutions).  
- Sensitivity analyses, to see how factor of safety changes with varying parameters.

Some key ideas from the referenced pages:

1. Kinematic and Mechanical Preconditions  
   - A slope can fail only if it is capable of forming a particular failure mechanism (kinematic condition)  
   - And if a trigger disturbs the force equilibrium (mechanical condition).

2. Plane Sliding vs. Rotational Sliding  
   - Plane sliding is when the failure surface is roughly planar (Fig. 10.31, 10.32).  
   - Rotational sliding is when the failure surface is curved, often occurring in cohesive soils (Fig. 10.34, 10.35).

3. Method of Slices  
   - We divide the potential sliding mass into vertical slices.  
   - Each slice’s forces (weight, cohesion, friction, pore-water pressures) are evaluated.  
   - Two approaches, referencing Fig. 10.37:  
     - Membrane solution: Groundwater is interpreted as acting along a membrane on the slip surface.  
     - Flow net solution: Uses equipotential lines and flow nets to apply seepage forces directly within each slice.

Below, you will find three main coding tasks, each preceded by explanatory Markdown cells that guide your analysis. This assignment is deliberately open-ended so you can adapt or extend the models to more complex scenarios.


In [1]:
# Part 1: Basic Plane Sliding
# We'll begin with a simple function to estimate a factor of safety (FOS) for plane sliding.
# This function is intentionally simplified. Refer back to Figures 10.31 and 10.32 in the textbook
# for details about how to incorporate seepage forces, partial saturation, etc.

import numpy as np

def plane_sliding_fos(phi_deg, beta_deg):
    """
    Computes a very simplified factor of safety for plane sliding in non-cohesive soil.
    phi_deg : friction angle (degrees)
    beta_deg: slope angle (degrees)
    Returns : factor of safety (dimensionless)
    """
    phi = np.radians(phi_deg)
    beta = np.radians(beta_deg)

    # For demonstration: ratio of available friction to driving shear
    # This is NOT a full formula; adapt from the text's equations if desired.
    if np.tan(beta) == 0:
        return np.inf

    fos = np.tan(phi) / np.tan(beta)
    return fos

# Example usage:
phi_test = 30.0  # friction angle in degrees
beta_test = 25.0 # slope angle in degrees
fos_result = plane_sliding_fos(phi_test, beta_test)
print(f"Plane sliding FOS with phi={phi_test}°, beta={beta_test}°: {fos_result:.3f}")


Plane sliding FOS with phi=30.0°, beta=25.0°: 1.238


## Part 1 Instructions

1. Review how the equilibrium of forces is established for plane sliding in non-cohesive soil.  
   - Check Figures 10.31 and 10.32 for the polygon of forces.  
   - Account for friction angle φ and slope angle β.  
   - (Optionally) consider seepage forces if the groundwater table intersects the slope.

2. Modify the `plane_sliding_fos` function to incorporate additional parameters:
   - Buoyancy or seepage if the water table is parallel to the slope (Fig. 10.32).
   - External loads if relevant.

3. Discuss in a separate Markdown cell how your final formula differs from the simplified example above.


In [ ]:
# Part 2: Method of Slices (Membrane vs. Flow Net)
# For cohesive soils, we can implement a simplified method of slices approach.
# Reference Fig. 10.37 from the textbook. We'll build two functions:
#   - One using a "membrane" interpretation of groundwater
#   - One using a "flow net" approach with each slice having distinct pore pressures.

import numpy as np

def fos_slices_membrane(n_slices, slope_height, c, phi_deg, gamma_soil):
    """
    Very simplified demonstration of a 'membrane solution' approach.
    n_slices     : number of slices
    slope_height : total slope height (m)
    c            : cohesion (kN/m^2)
    phi_deg      : friction angle (degrees)
    gamma_soil   : soil unit weight (kN/m^3)
    Returns an approximate factor of safety.
    """
    phi = np.radians(phi_deg)

    # For demonstration: assume each slice is of equal width
    slice_width = 2.0  # arbitrary
    slice_heights = np.linspace(1, slope_height, n_slices)

    R_total = 0.0
    D_total = 0.0

    for h_slice in slice_heights:
        # Weight of slice (assuming some unit thickness)
        W = gamma_soil * slice_width * h_slice

        # Simplistic normal force
        N = W * np.cos(phi)

        # Shear resistance from cohesion plus friction
        S_res = c * slice_width + N * np.tan(phi)

        # Driving force
        D = W * np.sin(phi)

        R_total += S_res
        D_total += D

    if D_total == 0:
        return np.inf
    return R_total / D_total


def fos_slices_flownet(n_slices, slope_height, c, phi_deg, gamma_soil):
    """
    Simplified demonstration of a 'flow net solution' approach.
    We'll add random or parametric pore water pressure for each slice.
    """
    phi = np.radians(phi_deg)
    slice_width = 2.0
    slice_heights = np.linspace(1, slope_height, n_slices)

    R_total = 0.0
    D_total = 0.0

    rng = np.random.default_rng(seed=42)  # reproducible random values

    for h_slice in slice_heights:
        W = gamma_soil * slice_width * h_slice

        # Assign a random pore pressure ratio in [0, 0.3]
        ru = rng.random() * 0.3

        # Effective normal force
        N_eff = (W * np.cos(phi)) * (1 - ru)

        # Shear resistance
        S_res = c * slice_width + N_eff * np.tan(phi)

        # Driving force
        D = W * np.sin(phi)

        R_total += S_res
        D_total += D

    if D_total == 0:
        return np.inf
    return R_total / D_total

# Demonstration usage:
fos_membrane_demo = fos_slices_membrane(n_slices=5, slope_height=10.0, c=10.0, phi_deg=25.0, gamma_soil=18.0)
fos_flownet_demo = fos_slices_flownet(n_slices=5, slope_height=10.0, c=10.0, phi_deg=25.0, gamma_soil=18.0)

print(f"Membrane solution FOS (example): {fos_membrane_demo:.3f}")
print(f"Flow net solution FOS (example): {fos_flownet_demo:.3f}")


## Part 2 Instructions

1. Analyze the functions `fos_slices_membrane` and `fos_slices_flownet`:
   - How are they currently summing the forces among slices?
   - Where can you incorporate real data or a more rigorous approach (e.g., geometry of the slip surface, slice base angles, etc.)?

2. Revise one or both functions to reflect your course’s recommended method for computing the factor of safety.
   - You might need to include more realistic geometry or moment equilibrium.  
   - Alternatively, you can keep it simple but justify your assumptions.

3. Document how your approach corresponds to the textbook’s Figures 10.37 and 10.38 (membrane vs. flow net solutions, interacting blocks, etc.).


In [ ]:
# Part 3: Sensitivity Analysis & Visualization
# Let's conduct a short sensitivity analysis by varying slope angle or pore pressure ratio,
# compute factor of safety, and then plot the results.

import numpy as np
import matplotlib.pyplot as plt

def plane_sliding_sensitivity(phi_deg, slope_angles):
    """
    Returns a list of factor of safety values for an array of slope angles
    using the plane_sliding_fos function from Part 1.
    """
    fos_list = []
    for beta in slope_angles:
        fos_val = plane_sliding_fos(phi_deg, beta)
        fos_list.append(fos_val)
    return fos_list

slope_angles_array = range(10, 46, 5)  # from 10 to 45 in steps of 5
phi_demo = 30.0
fos_vs_beta = plane_sliding_sensitivity(phi_demo, slope_angles_array)

plt.figure(figsize=(6,4))
plt.plot(list(slope_angles_array), fos_vs_beta, 'o-', label="Plane Sliding FOS")
plt.axhline(y=1.0, color='r', linestyle='--', label="FOS = 1.0")
plt.xlabel("Slope Angle (degrees)")
plt.ylabel("Factor of Safety")
plt.title(f"Plane Sliding FOS vs. Slope Angle (phi={phi_demo}°)")
plt.grid(True)
plt.legend()
plt.show()


## Part 3 Instructions

1. Experiment with the above sensitivity analysis:
   - Vary angles, friction angles, cohesion, or random pore pressure parameters.
   - Compare how quickly your slope’s FOS drops below 1.0.

2. Visualize your final results:
   - Possibly create multiple subplots for different conditions (e.g., dry vs. saturated slope).
   - Show how the FOS changes with each parameter and discuss your findings.

3. Write a brief conclusion in another Markdown cell:
   - Summarize the main insights from your numerical experiments.
   - Reflect on how well these simplified models approximate real slope behavior.

### Submission Notes

- Upload your final notebook to the class repository.
- Ensure that each part (plane sliding, method of slices, and sensitivity analysis) is fully documented.
- Discuss any real-world complexities (non-homogeneous soils, layered strata, seismic loads, etc.) that are simplified here.
- Reference the relevant textbook pages (Figs. 10.31–10.38) wherever applicable.

End of Assignment
